# **Inferência**

## **Baixe os arquivos .pkl e carrege no colab**

- **[modelo_sono_rf.pkl](https://drive.google.com/file/d/1q8Wle2pla9_nx8SrRf3AkCjabX8f9_mw/view?usp=sharing)**
- **[metricas_normalizacao.pkl](https://drive.google.com/file/d/1ioFgVQbLloX7e1hzRotbU4h_bAX_efST/view?usp=sharing)**

*obs: Os nomes dos arquivos devem estar iguais aos nomes referênciados acima*

## **Importando modelo e métricas**

In [28]:
import joblib
import pandas as pd

# 1. Carregar os artefatos
modelo = joblib.load('/content/modelo_sono_rf.pkl')
metricas = joblib.load('/content/metricas_normalizacao.pkl')

## **Tratando e recebendo os dados**

In [29]:
def preparar_dados_para_modelo(dados_front):
    # Mapeamentos (Texto -> Numérico/Coluna)
    mapa_bmi = {"Normal": 0, "Sobrepeso": 1, "Obeso": 2}
    mapa_profissoes = {
        "Enfermeiro(a)": "Occ_Nurse",
        "Médico(a)": "Occ_Doctor",
        "Engenheiro(a)": "Occ_Engineer",
        "Advogado(a)": "Occ_Lawyer",
        "Professor(a)": "Occ_Teacher",
        "Contador(a)": "Occ_Accountant",
        "Vendedor(a)": "Occ_Salesperson"
    }

    # Dados base numéricos
    dados_modelo = {
        'Age': dados_front['idade'],
        'BMI Category': mapa_bmi.get(dados_front['bmi'], 0),
        'Systolic': dados_front['sistole'],
        'Diastolic': dados_front['diastole'],
        'Stress Level': dados_front['stress'],
        'Sleep Duration': dados_front['sono_duracao'],
        'Heart Rate': dados_front['freq_cardiaca'],
        'Physical Activity Level': dados_front['ativ_fisica'],
        'Gender_Male': 1 if dados_front['genero'] == 'Homem' else 0,
        'Gender_Female': 1 if dados_front['genero'] == 'Mulher' else 0
    }

    # One-Hot Encoding manual da profissão
    prof_key = mapa_profissoes.get(dados_front['ocupacao'])
    if prof_key:
        dados_modelo[prof_key] = 1

    return dados_modelo

### **Dados do Paciente**

In [30]:
dados_paciente = {
            #Opções: "Homem", "Mulher"
            "genero": "Homem",
            #Opções: "Médico(a)", "Professor(a)", "Enfermeiro(a)", "Engenheiro(a)", "Contador(a)", "Advogado(a)", "Vendedor(a)",
            "ocupacao": "Professor(a)",
            #Opções: "Normal", "Sobrepeso", "Obeso"
            "bmi": "Obeso",
            "idade": 40,
            "ativ_fisica": 65,
            "freq_cardiaca": 90,
            "sistole": 140,
            "diastole": 120,
            "sono_duracao": 5.2,
            "stress": 6
        }

In [31]:
dados_tratados = preparar_dados_para_modelo(dados_paciente)

## **Executar inferência**

In [32]:
def classificar_risco(dados_paciente):
    if modelo is None:
        return {"Erro": 0.0, "Modelo não carregado": 0.0}

    # 1. Preparar e criar DataFrame
    dados_prontos = preparar_dados_para_modelo(dados_paciente)
    df = pd.DataFrame([dados_prontos])

    # 2. Normalização
    try:
        for col in metricas['numeric_cols']:
            if col in df.columns:
                df[col] = (df[col] - metricas['norm'][col]['mean']) / metricas['norm'][col]['std']
    except Exception as e:
        print(f"Aviso na normalização: {e}")

    # 3. Reindexar e Predizer
    df = df.reindex(columns=metricas['columns_order'], fill_value=0)
    probs = modelo.predict_proba(df)[0]

    # 4. Formatar Saída
    classes = {0: "Saudável (Sem Distúrbio)", 1: "Apneia do Sono", 2: "Insônia"}
    resultado_bruto = {classes[i]: prob for i, prob in enumerate(probs)}

    return dict(sorted(resultado_bruto.items(), key=lambda item: item[1], reverse=True))


In [33]:
resultado = classificar_risco(dados_paciente)

for doenca, porcentagem in resultado.items():
  print(f"{doenca}: {porcentagem:.2f}%")


Insônia: 0.46%
Apneia do Sono: 0.43%
Saudável (Sem Distúrbio): 0.11%


## **Formulário clínico**

Além desse colab de inferência, temos o MVP/PoC da nossa startup. No PoC a experiência é muita mais simples e fluída, por favor não deixe de conferir!

Nosso Poc: **[link](https://sleep-disorder-screening-deploy-vuqhvnqp9e2jjvl5tghs7u.streamlit.app/)**